In [ ]:
import os, sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Load Data

In [ ]:

directory = 'INSERT DIRECTORY' # Directory to a folder containing DataFrame generated by BJ_formatting.ipynb
file = 'INSERT FILENAME'
all_df = pd.read_pickle(os.path.join(directory, file)).reset_index(drop=True)
all_df.head()

# Filter Data

In [ ]:
# Constants
samples = 10 # Window size for sampling tail for alignment 

noise_threshold = -3.0 # Tail Noise Upper Limit

logG_high, logG_low = -0.5, -6.0 # Trace average bounds
Z_high, Z_low = 30E-3, 0E-3 # Plateau length bounds

zero_mark = -0.1 # THreshold for marking break point

In [ ]:
# Initialise pass boolean
all_df['passed'] = 1
all_df.head()

In [ ]:
filter_df = all_df.copy()

In [ ]:
# Test Plateau Length Null
num_nans = len(filter_df.loc[pd.isnull(filter_df.PlateauLen.values)])
print('Failed: {}'.format(num_nans))
filter_df = filter_df.loc[~pd.isnull(filter_df.PlateauLen.values)]

In [ ]:
# Test Plateau Length
plats = filter_df.PlateauLen.values
locs = (plats > Z_high) | (plats < Z_low) & (filter_df.passed == 1)
print('failed: {}'.format(len(filter_df.loc[locs])))
filter_df.loc[locs, 'passed'] = 0

In [ ]:
# Test mean of trace
means = np.vstack(filter_df.logG.values).mean(axis=1)
locs = (means > logG_high) | (means < logG_low) & (filter_df.passed == 1)
print('failed: {}'.format(len(filter_df.loc[locs])))
filter_df.loc[locs, 'passed'] = 0
filter_df.head()

In [ ]:
# Test Noise Tail
tail_means = np.vstack(filter_df.logG.values)[:, -samples:].mean(axis=1)
locs = (tail_means > noise_threshold) & (filter_df.passed == 1)
print('failed: {}'.format(len(filter_df.loc[locs])))
filter_df.loc[locs, 'passed'] = 0
filter_df.head()

In [ ]:
# Align Data
def align(row):
    trace = row.logG
    for i in range(len(trace) - samples):
        val = trace[i : i+samples].mean()
        if val < zero_mark:
            zero_idx = int(i + (samples / 2))
            break
    
    return row.Z - row.Z[zero_idx]

filter_df.loc[:, 'Z'] = filter_df.apply(align, axis=1)
filter_df.head()

# Save Clean Data

In [ ]:
filter_df.to_pickle(os.path.join(directory, "INSERT FILENAME"))